In [2]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.oauth2 import service_account
from apiclient.discovery import build
import warnings 
warnings.filterwarnings('ignore')

import mysql.connector
from pyspark.sql import SparkSession

In [3]:
def get_service(api_name, api_version, scopes, key_file_location):
    
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
            key_file_location, scopes=scopes)

    # Build the service object.
    service = build(api_name, api_version, credentials=credentials)

    return service
    
    
# Define the auth scopes to request.
scope = 'https://www.googleapis.com/auth/analytics.readonly'
key_file_location = './1111.json'

# Authenticate and construct service.
service = get_service(
        api_name='analytics',
        api_version='v3',
        scopes=[scope],
        key_file_location=key_file_location)
        
# Get a list of all Google Analytics accounts for this user
accounts = service.management().accounts().list().execute()

if accounts.get('items'):
    # Get the first Google Analytics account.
    account = accounts.get('items')[0].get('id')

    # Get a list of all the properties for the first account.
    properties = service.management().webproperties().list(accountId=account).execute()

In [5]:
# 참고 : https://ga-dev-tools.web.app/query-explorer/

result = service.data().ga().get(
            ids='ga:1111111',
           # max_results= None,
            max_results=30000,
            start_date='2022-05-01',
            end_date='2022-05-31',
            filters='ga:landingPagePath=@codeshare,ga:landingPagePath=@talkboard,ga:landingPagePath=@forum;ga:landingPagePath!@myprofile;ga:landingPagePath!=/community/codeshare',
            metrics='ga:users',
            dimensions='ga:landingPagePath,ga:sourceMedium').execute()
codeshare = pd.DataFrame(result['rows'], 
             columns=[col['name'] for col in result['columnHeaders']] )
codeshare['ga:users'] = codeshare['ga:users'].astype(int)
codeshare

,ga:landingPagePath,ga:sourceMedium,ga:users
0,/codeshare/1013,(direct) / (none),1
1,/codeshare/1019,google / organic,6
2,/codeshare/1029,google / organic,2
3,/codeshare/1029?dtype=vote&s_id=0,(direct) / (none),1
4,/codeshare/1030?dtype=recent,google / organic,1
...,...,...,...
8298,/m/competitions/official/235616/codeshare/1427,google / organic,1
8299,/m/competitions/official/235680/talkboard/,(direct) / (none),3
8300,/m/competitions/open/235560/codeshare/847/,(direct) / (none),2
8301,/search?q=cache:VEn_VbQPwn0J:https://dacon.io/...,google / organic,1


In [6]:
# 참고 : https://ga-dev-tools.web.app/query-explorer/

result = service.data().ga().get(
            ids='ga:111111',
           # max_results= None,
            max_results=30000,
            start_date='2022-05-01',
            end_date='2022-05-31',
            filters='ga:landingPagePath=@codeshare;ga:landingPagePath!@myprofile;ga:landingPagePath!=/community/codeshare',
            metrics='ga:users',
            dimensions='ga:landingPagePath,ga:sourceMedium').execute()
codeshare = pd.DataFrame(result['rows'], 
             columns=[col['name'] for col in result['columnHeaders']] )
codeshare['ga:users'] = codeshare['ga:users'].astype(int)
codeshare

,ga:landingPagePath,ga:sourceMedium,ga:users
0,/codeshare/1013,(direct) / (none),1
1,/codeshare/1019,google / organic,6
2,/codeshare/1029,google / organic,2
3,/codeshare/1029?dtype=vote&s_id=0,(direct) / (none),1
4,/codeshare/1030?dtype=recent,google / organic,1
...,...,...,...
6182,/en/competitions/open/235713/codeshare/3119?pa...,google / organic,1
6183,/m/codeshare/1720,(direct) / (none),1
6184,/m/competitions/official/235616/codeshare/1427,google / organic,1
6185,/m/competitions/open/235560/codeshare/847/,(direct) / (none),2


In [7]:
# 코드쉐어 아이디만 추려서 열로 만들기 
import re 
text = '/competitions/official/235747/codeshare/3039?p...'
def prep(text):
    try:
        answer = re.findall('codeshare/\d+', text)[0].split('/')[-1]
    except:
        answer = np.nan
    return answer
codeshare['cs_id'] = codeshare['ga:landingPagePath'].apply(prep) 
codeshare = codeshare.dropna()

In [8]:
codeshare

,ga:landingPagePath,ga:sourceMedium,ga:users,cs_id
0,/codeshare/1013,(direct) / (none),1,1013
1,/codeshare/1019,google / organic,6,1019
2,/codeshare/1029,google / organic,2,1029
3,/codeshare/1029?dtype=vote&s_id=0,(direct) / (none),1,1029
4,/codeshare/1030?dtype=recent,google / organic,1,1030
...,...,...,...,...
6182,/en/competitions/open/235713/codeshare/3119?pa...,google / organic,1,3119
6183,/m/codeshare/1720,(direct) / (none),1,1720
6184,/m/competitions/official/235616/codeshare/1427,google / organic,1,1427
6185,/m/competitions/open/235560/codeshare/847/,(direct) / (none),2,847
